# 02/15/23

Builds a ranking set from a fragmented/split datasets (train/test/val) (feature features)

Notes on 2024/1/16: the script is prviously used generate ranking set of r2-6144fp. Now  hyun-fp 

Notes on 2024/2/8:
Now generating both fp with cleaned dataset (cleaned by inchi)

Notes on 2024/2/22:
Now generate r2-r10 fp, still in the cleaned dataset

In [2]:
import torch, numpy as np, os, pickle, tqdm
from pathlib import Path

In [2]:
# 

In [3]:
folder_base = Path("/workspace/SMILES_dataset")
choice_split = ["val", "test"]

output_path = Path("/workspace/ranking_sets_cleaned_by_inchi")
os.makedirs(output_path, exist_ok=True)
# fp_paths = [ f"R{i}-6144FP" for i in range(3,11)]
# fp_paths+=[]
fp_paths = os.listdir(folder_base / "test")

fp_paths = list(filter(lambda x: "FP"  in x , fp_paths))
fp_paths

['R2-6144FP', 'HYUN_FP', 'R0_to_R4_reduced_FP']

In [4]:
import pickle

In [5]:
# fp_paths = ['R2-6144-count-based-FP']


In [1]:

for fp_path in fp_paths:
    for choice in choice_split:
    # tempdata/SMILES_dataset/<split>
        # STEP1: open pickle file (dict) to know all kinds of smiles
        index_to_smiles = pickle.load(open(f'/workspace/SMILES_dataset/{choice}/SMILES/index.pkl', 'rb'))
        
        choice_path = folder_base / choice / fp_path
        print(f"=== {choice_path}")

        if "index.pkl" in os.listdir(choice_path):
            print("\tGenerating ranking set from pickle index...")
            raise NotImplementedError("Building Ranking Set from index file not supported...")
        else:
            print("\tGenerating from list of files...")
            all_FP_indices_in_folder = os.listdir(choice_path)
            all_added_smiles = set()
            all_FP_indices_to_be_added = []
            for i in all_FP_indices_in_folder:
                smile = index_to_smiles [int(i.split(".")[0])]
                if smile not in all_added_smiles:
                    all_added_smiles.add(smile)
                    all_FP_indices_to_be_added.append(i)
            
            files = [torch.load(choice_path / file) for file in tqdm.tqdm(sorted(all_FP_indices_to_be_added))]
            out = torch.vstack(files)
            out_path = output_path  / f"SMILES_{fp_path}_ranking_sets" / choice
            os.makedirs(out_path, exist_ok=True)
            torch.save(out, out_path / "rankingset.pt")

NameError: name 'choice_split' is not defined

In [5]:
fp_path = "R0_to_R4_reduced_FP"
for choice in choice_split:
# tempdata/SMILES_dataset/<split>
    # STEP1: open pickle file (dict) to know all kinds of smiles
    
    choice_path = folder_base / choice / fp_path
    print(f"=== {choice_path}")

    if "index.pkl" in os.listdir(choice_path):
        print("\tGenerating ranking set from pickle index...")
        raise NotImplementedError("Building Ranking Set from index file not supported...")
    else:
        print("\tGenerating from list of files...")
        all_FP_indices_to_be_added = pickle.load(open(f'/root/MorganFP_prediction/reproduce_previous_works/smart4.5/datasets/{choice}_indices_of_full_info_NMRs.pkl', 'rb'))
        
        files = [torch.load(choice_path / file) for file in tqdm.tqdm(sorted(all_FP_indices_to_be_added))]
        out = torch.vstack(files)
        out_path = output_path  / f"SMILES_{fp_path}_ranking_sets_only_all_info_molecules" / choice
        os.makedirs(out_path, exist_ok=True)
        torch.save(out, out_path / "rankingset.pt")

=== /workspace/SMILES_dataset/val/R0_to_R4_reduced_FP
	Generating from list of files...


100%|██████████| 5657/5657 [00:05<00:00, 1000.45it/s]


=== /workspace/SMILES_dataset/test/R0_to_R4_reduced_FP
	Generating from list of files...


100%|██████████| 5463/5463 [00:03<00:00, 1792.53it/s]


In [22]:
k

tensor([0., 0., 0.,  ..., 0., 0., 0.])